## 1. 10 самых продаваемых видеоигр
<p>Видеоигры — это большой бизнес: по прогнозам, к 2027 году мировой игровой рынок будет стоить более 300 миллиардов долларов. Поскольку на кону так много денег, крупные издатели игр очень заинтересованы в создании следующего большого хита. Но становятся ли игры лучше или золотой век видеоигр уже прошел?</p>
<p>В этом проекте я рассмотрю 400 самых продаваемых видеоигр, созданных в период с 1977 по 2020 год. Я сравню данные о продажах игр с отзывами критиков и пользователей, чтобы определить, улучшились ли видеоигры с течением времени.</p>
<p>Наша база данных содержит две таблицы по 400 строк.
<h3 id="game_sales"><code>game_sales</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>Значение</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>game</code></td>
<td>varchar</td>
<td>Название игры</td>
</tr>
<tr>
<td style="text-align:left;"><code>platform</code></td>
<td>varchar</td>
<td>Игровая платформа</td>
</tr>
<tr>
<td style="text-align:left;"><code>publisher</code></td>
<td>varchar</td>
<td>Издатель игры</td>
</tr>
<tr>
<td style="text-align:left;"><code>developer</code></td>
<td>varchar</td>
<td>Разработчик игры</td>
</tr>
<tr>
<td style="text-align:left;"><code>games_sold</code></td>
<td>float</td>
<td>Количество проданных копий (миллионы)</td>
</tr>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Год релиза</td>
</tr>
</tbody>
</table>
<h3 id="reviews"><code>reviews</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>значение</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>game</code></td>
<td>varchar</td>
<td>Название игры</td>
</tr>
<tr>
<td style="text-align:left;"><code>critic_score</code></td>
<td>float</td>
<td> Оценка критиков, основываясь на Metacritic</td>
</tr>
<tr>
<td style="text-align:left;"><code>user_score</code></td>
<td>float</td>
<td> Оценка игроков, основываясь на Metacritic</td>
</tr>
</tbody>
</table>
<p>Давайте начнем с рассмотрения некоторых из самых продаваемых видеоигр всех времен!</p>

In [32]:
%%sql
postgresql:///games
-- Давайте найдем десять самых продаваемых видеоигр в game_sales.

-- Выбераем всю информацию для десяти самых продаваемых игр
Select *
from game_sales
group by game
-- Мы упорядочиваем результаты от самой продаваемой игры до десятой по популярности.
order by games_sold desc
limit 10

10 rows affected.


game,platform,publisher,developer,games_sold,year
Wii Sports for Wii,Wii,Nintendo,Nintendo EAD,82.90,2006
Super Mario Bros. for NES,NES,Nintendo,Nintendo EAD,40.24,1985
Counter-Strike: Global Offensive for PC,PC,Valve,Valve Corporation,40.00,2012
Mario Kart Wii for Wii,Wii,Nintendo,Nintendo EAD,37.32,2008
PLAYERUNKNOWN'S BATTLEGROUNDS for PC,PC,PUBG Corporation,PUBG Corporation,36.60,2017
Minecraft for PC,PC,Mojang,Mojang AB,33.15,2010
Wii Sports Resort for Wii,Wii,Nintendo,Nintendo EAD,33.13,2009
Pokemon Red / Green / Blue Version for GB,GB,Nintendo,Game Freak,31.38,1998
New Super Mario Bros. for DS,DS,Nintendo,Nintendo EAD,30.80,2006
New Super Mario Bros. Wii for Wii,Wii,Nintendo,Nintendo EAD,30.30,2009


## 2. Отсутствующие данные
<p>Как вы видете, самые продаваемые видеоигры были выпущены в период с 1985 по 2017 год! Это довольно большой диапазон; нам придется использовать данные из таблицы <code>reviews</code>, чтобы получить более полное представление о лучших годах для видеоигр. </p>
<p>Во-первых, важно изучить ограничения нашей базы данных. Большим недостатком является отсутствие данных в <code>reviews</code> для некоторых игр в таблице <code>game_sales</code>. Нужно учесть это  </p>

In [34]:
%%sql 
--Давайте определим, у скольких игр в таблице game_sales отсутствуют как user_score, так и сritic_score.

-- Выберем количество игр, в которых значения Critic_score и User_score равны нулю, то есть null.
Select count(*)
from game_sales as g
-- Соединим games_sales и reviews с помощью left join
LEFT JOIN reviews as r 
on g.game=r.game
where critic_score is null and user_score is null


 * postgresql:///games
1 rows affected.


count
31


## 3. Годы, когда игры понравились критикам 
<p>Похоже, что чуть менее десяти процентов игр в таблице <code>game_sales</code> не содержат данных об отзывах. Это достаточно небольшой процент, чтобы мы могли продолжить исследование, но об отсутствующих данных в reviews полезно помнить, когда мы будем оценивать результаты более сложных запросов SQL. </p>
<p>Существует множество способов определить лучшие годы для видеоигр. Начнем с того, что на отзывы критиков. </p>

In [36]:
%%sql
-- Давайте найдем годы с самым высоким средним рейтингом критиков.
-- Выберем год выпуска (year) из таблицы game_sales и среднюю оценку критиков для каждого года из таблицы reviews.
select g.year,
    round(avg(r.critic_score),2) as avg_critic_score
from game_sales as g
-- соединим с помощью inner join таблицы Reviews и game_sales
inner join reviews as r    
on g.game=r.game
-- сгруппируем результаты по году выпуска
group by g.year
-- Упорядочим данные от самого высокого до самого низкого avg_critic_score и ограничем результаты до 10.
order by avg_critic_score desc
limit 10;

 * postgresql:///games
10 rows affected.


year,avg_critic_score
1990,9.80
1992,9.67
1998,9.32
2020,9.20
1993,9.10
1995,9.07
2004,9.03
1982,9.00
2002,8.99
1999,8.93


## 4. Был ли 1982 год действительно таким успешным?
<p>По отзывам критиков, диапазон успешных лет составляет от 1982 по 2020 год, мы не приблизились к нашему желанному результату. </p>
<p>Если посмотреть по-внимательнее, то мы видим, что некоторые из значений <code>avg_critic_score</code> выглядят подозрительно, имея круглые средние значения. Результат за 1982 год выглядит особенно подозрительным. Возможно, в нашем наборе данных было не так много видеоигр, выпущенных в некоторые годы, поэтому средняя для этих лет выглядит подозрительно. </p>
<p>Давайте обновим наш запрос и выясним, действительно ли 1982 год был таким удачным для видеоигр, как уверяют результаты по средней оценке критиков. </p>

In [38]:
%%sql 
-- Вставим запрос из предыдущей задачи; обновим его, добавив количество игр, выпускаемых каждый год, 
-- под названием num_games
select g.year,
    round(avg(r.critic_score),2) as avg_critic_score,
    count(g.game) as num_games
from game_sales as g
inner join reviews as r    
on g.game=r.game
group by g.year
-- Обновим запрос, чтобы он возвращал только те годы, в которых количество,
-- оцененных критиками видеоигр было больше четырех
Having count(g.game)>4
order by avg_critic_score desc
limit 10;

 * postgresql:///games
10 rows affected.


year,avg_critic_score,num_games
1998,9.32,10
2004,9.03,11
2002,8.99,9
1999,8.93,11
2001,8.82,13
2011,8.76,26
2016,8.67,13
2013,8.66,18
2008,8.63,20
2017,8.62,13


## 5. Годы, выпавшие из списка фаворитов критиков
<p> Столбец <code> num_games</code> указывает на то, что наш новый список лучших игр по мнению критиков отражает годы, когда было выпущено довольно много игр с хорошими рецензиями, а не всего лишь один или два хита. Но какие годы выпали из списка из-за того, что критики оценили четыре или менее игр? Давайте определим их для интереса.</p>
<p> Я создала новые таблицы с данными из прошлых запросов</p>
<h3 id="top_critic_years"><code>top_critic_years</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>значение</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Год релиза </td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_critic_score</code></td>
<td>float</td>
<td>Средняя для всех значений отзывов критиков по играм выпущенным в данном году</td>
</tr>
</tbody>
</table>
<h3 id="top_critic_years_more_than_four_games"><code>top_critic_years_more_than_four_games</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Год релиза</td>
</tr>
<tr>
<td style="text-align:left;"><code>num_games</code></td>
<td>int</td>
<td>Количество игр, выпущенных в данном году</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_critic_score</code></td>
<td>float</td>
<td>Средняя для всех значений отзывов критиков по играм выпущенным в данном году</td>
</tr>
</tbody>
</table>

In [40]:
%%sql 

-- Выберем год и avg_critic_score для тех лет, которые выпали из списка фаворитов критиков.
select year, avg_critic_score
from top_critic_years
except 
select year, avg_critic_score
from top_critic_years_more_than_four_games
-- Упорядочим результаты от самого высокого до самого низкого avg_critic_score
order by avg_critic_score desc;

 * postgresql:///games
6 rows affected.


year,avg_critic_score
1990,9.80
1992,9.67
2020,9.20
1993,9.10
1995,9.07
1982,9.00


## 6. Годы, полюбившиеся игрокам видеоигр
<p>Похоже, что начало 1990-х годов можно считать золотым веком видеоигр, основываясь только на <code>critic_score</code>, но нам нужно собрать больше данных об играх и отзыввах критиков для дальнейшего анализа. </p>
<p>Давайте перейдем к рассмотрению мнения другой важной группы людей, игроков. Для начала давайте создадим запрос, который будет рассматривать средние значения <code>user_score</code> по годам. </p >

In [42]:
%%sql 
-- Выберем год, округленное среднее значение user_score и количество игр, выпущенных в году.
select g.year,
    round(avg(r.user_score),2) as avg_user_score,
    count(g.game) as num_games
from game_sales as g
-- соединим две таблицы 
inner join reviews as r    
on g.game=r.game
-- сгруппироуем данные по годам
group by g.year
-- Включим только годы, в которых игроки оценили более четырех игр
Having count(g.game)>4
-- Упорядочим данные по avg_user_score и ограничим результаты до десяти.
order by avg_user_score desc
limit 10;

 * postgresql:///games
10 rows affected.


year,avg_user_score,num_games
1997,9.50,8
1998,9.40,10
2010,9.24,23
2009,9.18,20
2008,9.03,20
1996,9.00,5
2005,8.95,13
2006,8.95,16
2000,8.80,8
2002,8.80,9


## 7. Годы, полюбившиеся и критикам, и игрокам
<p>У нас есть список десяти лучших лет по отзывам критиков и пользователей. Есть ли годы, которые указаны в обеих таблицах? </p>
<p> Поработаем с таблицой <code>top_critic_years_more_than_four_games</code>, в которой хранятся результаты нашего запроса из задачи 4: </p>
<h3 id="top_critic_years_more_than_four_games"><code>top_critic_years_more_than_four_games</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>значение </th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Год релиза игры</td>
</tr>
<tr>
<td style="text-align:left;"><code>num_games</code></td>
<td>int</td>
<td>Количество выпущенных игр в данном году</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_critic_score</code></td>
<td>float</td>
<td>Средняя оценка по отзывам критиков видеоигр в данном году</td>
</tr>
</tbody>
</table>
<p>Я также сохранила результаты нашего запросa из предыдущей задачи в таблицу:</p>
<h3 id="top_user_years_more_than_four_games"><code>top_user_years_more_than_four_games</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>значение</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Год релиза игры</td>
</tr>
<tr>
<td style="text-align:left;"><code>num_games</code></td>
<td>int</td>
<td>Количество выпущенных игр в данном году</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_user_score</code></td>
<td>float</td>
<td>Средняя оценка по отзывам игроков в данном году</td>
</tr>
</tbody>
</table>

In [44]:
%%sql 

-- Выберем года, которые присутствуют в двух таблицах
select year
from top_critic_years_more_than_four_games
-- используем intersect, так как нам нужны года, которые есть в двух таблицах
intersect
select year
from top_user_years_more_than_four_games

 * postgresql:///games
3 rows affected.


year
1998
2008
2002


## 8. Продажи в лучшие годы видеоигр
<p> Похоже, что у нас есть три года, в течение которых были выпущены игры, которые по единодушному мнению пользователей и критиков, входят в топ 10 лучших. Есть много других способов определить, какие годы являются лучшими для видеоигр, но давайте пока остановимся на этих годах. Мы знаем, что эти годы понравились критикам и игрокам, но как насчет производителей видеоигр? Какой был показатель продаж? Давайте узнаем.</p>


In [46]:
%%sql 
--Мы будем использовать запрос из предыдущей задачи в качестве подзапроса в этой.

-- Выберем год и общую games_sold (total_games_sold)
select year,
    sum(games_sold) as total_games_sold
from game_sales
-- Используем фильтр game_sales на основе того, присутствует ли каждый год в списке, 
-- возвращенном в предыдущей задаче. 
-- Запрос из предыдущей задачи используем как подзапрос.
where year in (select year
    from top_critic_years_more_than_four_games
    intersect
    select year
    from top_user_years_more_than_four_games)
group by year
-- упорядочим результаты по убыванию значения total_games_sold
order by total_games_sold desc

 * postgresql:///games
3 rows affected.


year,total_games_sold
2008,175.07
1998,101.52
2002,58.67


In [ ]:
## 9. Результат 
<p> Похоже, что самыми успешными годами были 2002, 2008, и 1998. Годом, в котором было наибольшее количество продаж, 
довольных критиков и игроков является 2008 /p>
